In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/30daysofml/sample_submission.csv
/kaggle/input/30daysofml/train.csv
/kaggle/input/30daysofml/test.csv


In [2]:
#loading the data

train = pd.read_csv('/kaggle/input/30daysofml/train.csv',index_col="id")
test = pd.read_csv('/kaggle/input/30daysofml/test.csv',index_col="id")
train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,
1,B,B,B,C,B,B,A,E,C,N,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
2,B,B,A,A,B,D,A,F,A,O,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
3,A,A,A,C,B,D,A,D,A,F,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
4,B,B,A,C,B,D,A,E,C,K,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
6,A,A,A,C,B,D,A,E,A,N,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [3]:
#Counting the number of unique values in each categorical column
categorical_cols=['cat'+str(i) for i in range(10)]

categorical_nunique = list(map(lambda col: train[col].nunique(), categorical_cols))
d = dict(zip(categorical_cols, categorical_nunique))

sorted(d.items(), key=lambda x: x[1])

[('cat0', 2),
 ('cat1', 2),
 ('cat2', 2),
 ('cat3', 4),
 ('cat4', 4),
 ('cat5', 4),
 ('cat8', 7),
 ('cat6', 8),
 ('cat7', 8),
 ('cat9', 15)]

No column has more than 15 unique values, so we will use OneHotEncoding

In [4]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[categorical_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(test[categorical_cols]))

OH_cols_train.index = train.index
OH_cols_test.index = test.index

num_X_train = train.drop(categorical_cols, axis=1)
num_X_test = test.drop(categorical_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

In [5]:
train = OH_X_train
test = OH_X_test
X = OH_X_train.drop(["target"],axis=1)
y = OH_X_train.target
columns = X.columns.to_list()

In [6]:
from sklearn.linear_model import ElasticNet, Lasso,LinearRegression,RidgeCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [7]:
parameters_xgb = {"n_estimators":10000,
    "learning_rate":0.03,
    "subsample":0.8,
    "colsample_bytree":0.1,
    "max_depth":3,
    "booster":'gbtree',
    "tree_method":'gpu_hist',
    "predictor":'gpu_predictor',
    "reg_lambda":0.0009,
    "reg_alpha":23,
    "random_state":42}

In [8]:
pred1 = np.zeros(train.shape[0])
pred2 = np.zeros(train.shape[0])
pred3 = np.zeros(train.shape[0])
pred4 = np.zeros(train.shape[0])

test1 = np.zeros(test.shape[0])
test2 = np.zeros(test.shape[0])
test3 = np.zeros(test.shape[0])
test4 = np.zeros(test.shape[0])

kf = KFold(n_splits=5,random_state=48,shuffle=True)
n=0

for trn_idx, test_idx in kf.split(train[columns],train['target']):
    print(f"fold: {n+1}")
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]

    model1 = ElasticNet(alpha=0.001)
    model1.fit(X_tr,y_tr)
    pred1[test_idx] = model1.predict(X_val)
    test1 += model1.predict(test[columns])/kf.n_splits
    rmse1 = mean_squared_error(y_val, model1.predict(X_val), squared=False)
    print(">> model1 rmse = {}".format(rmse1))
    
    model2 = xgb.XGBRegressor(**parameters_xgb)
    model2.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=200,verbose=False)
    pred2[test_idx] = model2.predict(X_val)
    test2 += model2.predict(test[columns])/kf.n_splits
    rmse2 = mean_squared_error(y_val, model2.predict(X_val), squared=False)
    print(">> model2 rmse = {}".format(rmse2))
    
    model3 = LinearRegression()
    model3.fit(X_tr,y_tr)
    pred3[test_idx] = model3.predict(X_val)
    test3 += model3.predict(test[columns])/kf.n_splits
    rmse3 = mean_squared_error(y_val, model3.predict(X_val), squared=False)
    print(">> model3 rmse = {}".format(rmse3))
    
    model4 = RandomForestRegressor(n_estimators=50)
    model4.fit(X_tr,y_tr)
    pred4[test_idx] = model4.predict(X_val)
    test4 += model4.predict(test[columns])/kf.n_splits
    rmse4 = mean_squared_error(y_val, model4.predict(X_val), squared=False)
    print(">> model5 rmse = {}".format(rmse4))
    
    print(">> average all models rmse = {}".format((rmse1+rmse2+rmse3+rmse4)/4))
    
    n+=1

fold: 1
>> model1 rmse = 0.738281410919907
>> model2 rmse = 0.7170045226230315
>> model3 rmse = 0.7381433063992338
>> model5 rmse = 0.738491305376353
>> average all models rmse = 0.7329801363296312
fold: 2
>> model1 rmse = 0.7412231370966909
>> model2 rmse = 0.720703379314359
>> model3 rmse = 0.7411615105692857
>> model5 rmse = 0.7417714812737589
>> average all models rmse = 0.7362148770635236
fold: 3
>> model1 rmse = 0.7311930196302483
>> model2 rmse = 0.7114083466210702
>> model3 rmse = 0.7311802020705193
>> model5 rmse = 0.7314836968985812
>> average all models rmse = 0.7263163163051047
fold: 4
>> model1 rmse = 0.7393635928491521
>> model2 rmse = 0.7185732584000335
>> model3 rmse = 0.7394212503511077
>> model5 rmse = 0.7404265503298252
>> average all models rmse = 0.7344461629825295
fold: 5
>> model1 rmse = 0.74147447724337
>> model2 rmse = 0.7210756481779986
>> model3 rmse = 0.7414399580548736
>> model5 rmse = 0.7423268854244415
>> average all models rmse = 0.7365792422251709


In [9]:
stacked_predictions = np.column_stack((pred1,pred2,pred3,pred4))
stacked_test_predictions = np.column_stack((test1,test2,test3,test4))

kf = KFold(n_splits=5,random_state=42,shuffle=True)
final_prediction = np.zeros(test.shape[0])
rmse=[]  # list contains rmse for each fold
n=0
for trn_idx, test_idx in kf.split(stacked_predictions,train['target']):
    X_tr,X_val=stacked_predictions[trn_idx],stacked_predictions[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    
    meta_model = Lasso(alpha =0.00001)
    meta_model.fit(X_tr,y_tr)
    
    final_prediction +=meta_model.predict(stacked_test_predictions)/kf.n_splits
    rmse.append(mean_squared_error(y_val, meta_model.predict(X_val), squared=False))
    print(f"fold: {n+1}, rmse: {rmse[n]}")
    n+=1

    
np.mean(rmse)

fold: 1, rmse: 0.7169859531235715
fold: 2, rmse: 0.7170697840091459
fold: 3, rmse: 0.7190116401373366
fold: 4, rmse: 0.7187000027299999
fold: 5, rmse: 0.7170160641370964


0.71775668882743

In [10]:
submission = pd.DataFrame(data=final_prediction,index=test.index,columns=['target'])
submission

,target
id,
0,8.103876
5,8.394576
15,8.405846
16,8.543262
17,8.148112
...,...
499987,8.006784
499990,8.489190
499991,8.483515


In [11]:
submission.to_csv('submission.csv')